## Tecnologias e Aplicações
## Trabalho prático 2, Março 2020

### German Trafic Sign Recognition using CNN

Using the GTSR dataset for recognition of traffic signs, we created a convolutional neural network that recognizes traffic signs.

To improve the model the following elements were changed:
- batch size
- image size
- the number of layers (depth of the network):
    - Padding, and its size
    - number and size of filters for the convolution layers
    - How many dropouts, and its percentage
    - How many dense layers
- number of epochs

These elements are discussed below.

In [ ]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np

import matplotlib.pyplot as plt
import pathlib
from PIL import Image
import IPython.display as display

In [ ]:
dataset_path = "gtsrb/"

### Batch size

Batch size is an important parameter when training a network. It can influence speed and generalization, not necessarily in the same direction. There is no golden rule for the batch size but 32 is a commom number to start with.

### Image size

Bigger images means more computation operations per layer as well as more memory requirements, but better filters. So we choose 64 x 64 images.

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = 64

### Prepare to load images

Getting all the class names in a numpy array.

In [ ]:
data_dir = pathlib.Path(dataset_path + "train_images/")
trainClassNames = np.array(os.listdir(data_dir))

data_dir = pathlib.Path(dataset_path + "val_images/")
valClassNames = np.array(os.listdir(data_dir))

print("training", trainClassNames)
print("validation", valClassNames)

### Auxiliary functions 

#### get_label
Gets the label of an image, through its path. This function returns an array of Booleans where all elements are False except one, and the position of that True represents a class.

#### decode_img

Process the image for better performance and usage, by converting to uint8, making it binary and resizing.

#### get_bytes_and_label

Receives a path for an image and returns the image and its label.

In [ ]:
def get_label(file_path):
  # convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  return parts[-2] == classNames

def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_png(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMAGE_SIZE,IMAGE_SIZE])

def get_bytes_and_label(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  return img, label

### Loading images

The images are loaded to a Dataset from Keras and shuffled.

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

listset = tf.data.Dataset.list_files(dataset_path + "train_images/*/*.png")
train_dataset = listset.map(get_bytes_and_label, num_parallel_calls = AUTOTUNE)

listset = tf.data.Dataset.list_files(dataset_path + "val_images/*/*.png")
val_dataset = listset.map(get_bytes_and_label, num_parallel_calls = AUTOTUNE)

listset = tf.data.Dataset.list_files(dataset_path + "test_images/*/*.png")
test_dataset = listset.map(get_bytes_and_label, num_parallel_calls = AUTOTUNE)

### Train set and validation set

Due to the way the GTSRB is built (using video sequences) the set needs to be partitioned manually as to keep the sequences together in either set. This partition must guarantee that both sets contain an even amount of all classes.

### Information about image shape and size of training set

In [ ]:
for image, label in train_dataset.take(1):
  print("Image shape: ", image.numpy().shape)
  
dataset_length = [i for i,_ in enumerate(train_dataset)][-1] + 1
print("Total images in dataset: ",dataset_length)


### Data augmentation processing function

In [ ]:
import random

#https://www.tensorflow.org/api_docs/python/tf/image
#https://www.tensorflow.org/addons/api_docs/python/tfa/image

#pip install tensorflow-addons
import tensorflow_addons as tfa

def process_image(image, label):
    image = tf.image.resize(image, (64,64))
    
    # Devolde um numero aleatorio, atraves de uma distribuição normal, de -0.25 até 0.249..9
    r = tf.random.uniform(shape=(), minval=0, maxval=1)* 0.5 - 0.25
    
    # Aplica uma rotação no sentido contrario dos relogios em radianos
    # ou seja, aplica uma rotação de entre -14.324 a 14.324
    image = tfa.image.rotate(image, r)
    
    # Devolde um numero de -10 até 9.(9)
    rx = tf.random.uniform(shape=(), minval=0, maxval=1) * 20 - 10
    # Devolde um numero de -4 até 3.(9)
    ry = tf.random.uniform(shape=(), minval=0, maxval=1) * 8 - 4
    # Aplica uma translação com o vetor [rx, ry]
    image = tfa.image.translate(image, [rx, ry])
    # Ajusta o hue, saturação e rgb
    # o hue é escolhido aleatoriamente num intervalo [-0.3, 0,3]
    # a saturaçao é escolhida aleatoriamente num intervalo [0.9, 1,1]
    # scale_value é escolhido aleatoriamente num intervalo [0.9, 1,3]
    image = tfa.image.random_hsv_in_yiq(image, 0.3, 0.9, 1.1, 0.9, 1.3)
    image = tf.image.resize(image, (32,32))
    
    # Aumenta a brightness com um valor aleatorio de [-0.2, 0.8[
    # Todos os ... que sejam abaixo de 0.1 passam a ser 0.1
    image = tf.clip_by_value(tf.image.adjust_brightness(image, tf.random.uniform(shape=(), minval=0, maxval=1)-0.2),0,1)
    return image, label

### Preparing datasets

The datasets are prepared for better performance.

In [ ]:
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(buffer_size = dataset_length)
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
train_dataset = train_dataset.map(process_image)
train_dataset = train_dataset.batch(batch_size=BATCH_SIZE)
train_dataset = train_dataset.repeat()

val_dataset = val_dataset.cache()
val_dataset = val_dataset.shuffle(buffer_size = dataset_length)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.map(process_image)
val_dataset = val_dataset.batch(batch_size=BATCH_SIZE)
val_dataset = val_dataset.repeat()

test_dataset = test_dataset.batch(batch_size=BATCH_SIZE)

### Show a batch of training images

In [ ]:
def show_batch(image_batch, label_batch):
  columns = 6
  rows = BATCH_SIZE / columns + 1  
  plt.figure(figsize=(10, 2 * rows))
  for n in range(BATCH_SIZE):
      ax = plt.subplot(rows, columns, n+1)
      plt.imshow((image_batch[n]))
      plt.title(classNames[label_batch[n]==1][0])
      plt.axis('off')
        
        
image_batch, label_batch = next(iter(train_dataset))        
show_batch(image_batch, label_batch.numpy())

#### Number of Layers
One good strategy is to add layers to the model util it starts to overfit.

#### Padding
Bigger padding implies that the neural network must learn that the padding isn't relevant for classification. So we used padding with size one because it's the smallest padding and this way some pixels from the image are not ignored.

#### Conv2D
For the Conv2D layers, the number of filters starts at 32 and goes up, doubling it every layer. This serves to catch fewer filters at begging and more filters at the end. The Neural Network finds some features that are common by some signs at the beginning and more features at the end making the neural network recognize each sign individually.

#### Dropouts
Dropouts are used to maintain randomness. It helps to overcome overfit.

#### Denses
There are two dense layers, one with 256 nodes and one with 43 nodes. The first one is used in association that can exist among any feature to any other feature in a data point. The second gives the class name of the image.

#### Model
The model consists of 4 major groups. The first three groups are equal and consist of the following sequence:
1. Conv2D
2. BatchNormalization
3. Dropout
4. Conv2D
5. BatchNormalization
6. MaxPooling2D
7. Dropout

The fourth group consist of two dense layers with a BatchNormalization and a Dropout between.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, ZeroPadding2D
from tensorflow.keras import metrics
from tensorflow.keras.optimizers import Adam

def cnn55D3L2FC(classCount, imgSize, channels):
    
    model = Sequential()
    
    model.add(ZeroPadding2D(padding=(1,1),input_shape=(imgSize, imgSize, channels)))
    
    model.add(Conv2D(32, (5, 5), padding='same',activation='relu'))                        
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (5, 5), padding='same',activation='relu'))                        
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (5, 5), padding='same',activation='relu'))                        
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    model.add(Conv2D(256, (5, 5), padding='same',activation='relu'))                        
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(512, (5, 5), padding='same',activation='relu'))                        
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(1024, (5, 5), padding='same',activation='relu'))                        
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    
    #Fully conected layer
    model.add(Flatten())
    
    #Three more layers
    model.add(Dense(256, activation = "relu")) #Fully connected layer
    model.add(BatchNormalization())
    model.add(Dropout(0.2))
    
    model.add(Dense(classCount, activation='softmax'))
   

    #opt = Adam(lr=0.001)
    model.compile(optimizer = "adam", loss='categorical_crossentropy', metrics=[ metrics.categorical_accuracy])
    
    return model

model = cnn55D3L2FC(43, 64, 3)


### Draw a diagram of the network

This requires installing some packages, namely graphviz

In [ ]:
tf.keras.utils.plot_model(model, 'multi_input_and_output_model.png', show_shapes=True)

### Display a table with model information

When building a model kee an eye on the number of trainable parameters. Try to keep it below 10 million

In [ ]:
print(model.summary())

### Train the network

#### **Number of epochs**
Similar to the number of layers, a good strategy is to increase the number of epochs until it starts to overfit.

In [ ]:
from keras.callbacks.callbacks import EarlyStopping

callbacks = [
    EarlyStopping(
            monitor='val_categorical_accuracy',
            min_delta=0,
            patience=2, #Este patience diz que se não melhorar em 1 epoch fodeu basicamente
            verbose=3,
            mode='max'
        )
    ]

history = model.fit(train_dataset, steps_per_epoch = dataset_length/BATCH_SIZE, epochs=20, validation_data = val_dataset)

#history = model.fit(dataset, steps_per_epoch = 0.8*dataset_length/BATCH_SIZE,
#          epochs=100, validation_data = val_dataset, validation_steps= 0.2*dataset_length/BATCH_SIZE)

### Plot the training history

In [ ]:
print(history.history.keys())

# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Evaluate the model on the test set

This is the accuracy number that really matters. The current accuracy is 98.2%.

In [ ]:
model.evaluate(test_dataset)